In [ ]:
import requests
import pandas as pd
import time

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# ===========================
# 🔧 CONFIGURATION
# ===========================

DOMAIN = os.getenv("PIPEDRIVE_COMPANY_DOMAIN")
API_TOKEN = os.getenv("PIPEDRIVE_API_TOKEN")

LIMIT = 500
OUTPUT_CSV = "pipedrive_persons_v1.csv"

# ===========================
# 📥 FETCH ALL PERSONS (v1 returns full info)
# ===========================
def fetch_all_persons():
    all_persons = []
    base_url = f"https://{DOMAIN}/api/v1/persons"
    start = 0

    print("Fetching all persons (v1, includes phones & emails)...")

    while True:
        params = {"api_token": API_TOKEN, "start": start, "limit": LIMIT}
        res = requests.get(base_url, params=params, timeout=30)
        if res.status_code != 200:
            print(f"❌ Error {res.status_code}: {res.text}")
            break

        data = res.json()
        if not data.get("success"):
            break

        persons = data.get("data", [])
        if not persons:
            break

        all_persons.extend(persons)
        print(f"Fetched {len(all_persons)} persons so far...")

        pagination = data.get("additional_data", {}).get("pagination", {})
        if not pagination.get("more_items_in_collection"):
            break

        start = pagination.get("next_start", 0)
        time.sleep(0.2)

    print(f"✅ Total persons fetched: {len(all_persons)}")
    return all_persons


# ===========================
# 💾 SAVE TO CSV
# ===========================
def save_to_csv(persons):
    if not persons:
        print("⚠️ No persons found.")
        return

    df = pd.json_normalize(persons)

    # Flatten phones and emails
    def extract_values(field):
        if isinstance(field, list):
            return ", ".join([f.get("value", "") for f in field if isinstance(f, dict)])
        return ""

    df["phones"] = df["phone"].apply(extract_values)
    df["emails"] = df["email"].apply(extract_values)

    if "org_id.name" in df.columns:
        df.rename(columns={"org_id.name": "organization_name"}, inplace=True)

    selected_cols = ["id", "name", "phones", "emails", "organization_name", "owner_id.name", "add_time", "update_time"]
    df = df[[c for c in selected_cols if c in df.columns]]

    df.to_csv(OUTPUT_CSV, index=False, engine="openpyxl")
    print(f"📂 Saved to {OUTPUT_CSV}")


# ===========================
# 🚀 MAIN
# ===========================
if __name__ == "__main__":
    persons = fetch_all_persons()
    save_to_csv(persons)


Fetched 17000 persons so far...
Fetched 17500 persons so far...
Fetched 18000 persons so far...
Fetched 18500 persons so far...
Fetched 19000 persons so far...
Fetched 19500 persons so far...
Fetched 20000 persons so far...
Fetched 20500 persons so far...
Fetched 21000 persons so far...
Fetched 21500 persons so far...
Fetched 22000 persons so far...
Fetched 22500 persons so far...
Fetched 23000 persons so far...
